In [1]:
!pip install datasets transformers rouge_score --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 28.9 MB/s eta 0:00:00


In [2]:
!pip install accelerate>=0.21.0 --quiet


In [3]:
!pip install transformers[torch] --quiet


In [4]:
import accelerate
print(accelerate.__version__)


0.29.3


In [5]:
from google.colab import drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"Error mounting Google Drive: {e}")


Mounted at /content/drive


In [6]:
import pandas as pd
train_data = pd.read_csv('/content/drive/MyDrive/cnn_dailymail/train.csv')
validation_data = pd.read_csv('/content/drive/MyDrive/cnn_dailymail/validation.csv')
test_data = pd.read_csv('/content/drive/MyDrive/cnn_dailymail/test.csv')


In [7]:

"""###Data Preprocessing"""

#dropping ID column
train_data = train_data.drop(['id'], axis=1)
validation_data = validation_data.drop(['id'], axis=1)
test_data = test_data.drop(['id'], axis=1)

print(train_data.shape, validation_data.shape, test_data.shape)

train_data.head()

validation_data.head()

test_data.head()

"""### BERT2BERT"""

!pip install datasets
import datasets
from datasets import Dataset

!pip install transformers==4.2.1
import transformers
from transformers import BertTokenizerFast

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(validation_data)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

batch_size=8
encoder_max_length=512
decoder_max_length=128

def process_data_to_model_inputs(batch):

  inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()


  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

train_dataset = train_dataset.select(range(15000))

train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "highlights"]
)
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_dataset = val_dataset.select(range(200))

val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "highlights"]
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_dataset

from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

!pip install rouge_score


(287113, 2) (13368, 2) (11490, 2)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.2/184.2 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

In [8]:

"""### FINE-TUNING"""

rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions


    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    overwrite_output_dir=True,
    save_total_limit=3,

)

trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()



<ipython-input-8-526846fbc168>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
500,0.057900,0.000125,0.000000,0.000000,0.000000
1000,0.000400,0.000045,0.000000,0.000100,0.000100
1500,0.001000,0.000037,0.000000,0.000100,0.000100
2000,0.000700,0.000044,0.000000,0.000000,0.000000
2500,0.000100,0.000056,0.000000,0.000000,0.000000
3000,0.000100,0.000023,0.000000,0.000000,0.000000
3500,0.000300,0.000022,0.000000,0.000000,0.000000
4000,0.000100,0.000013,0.000000,0.000000,0.000000
4500,0.000000,0.000011,0.000000,0.000000,0.000000
5000,0.000000,0.000010,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decode

TrainOutput(global_step=5625, training_loss=0.005389767155134016, metrics={'train_runtime': 9079.2963, 'train_samples_per_second': 4.956, 'train_steps_per_second': 0.62, 'total_flos': 2.76054345216e+16, 'train_loss': 0.005389767155134016, 'epoch': 3.0})

In [9]:


"""###TESTING"""

!ls

dummy_bert2bert = EncoderDecoderModel.from_pretrained("./checkpoint-5500")

import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("./checkpoint-5500")
model.to("cuda")

test_dataset = Dataset.from_pandas(test_data)

test_dataset = test_dataset.select(range(200))

batch_size = 8

def generate_summary(batch):

    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)


    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_dataset.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]
label_str = results["highlights"]

rougeL_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid
rouge1_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
rouge2_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rougeL_output)
print(rouge1_output)
print(rouge2_output)

results[1]

pred_str

test_dataset['article'][1]


'=0.21.0'   checkpoint-4500   checkpoint-5000   checkpoint-5500   drive   runs	 sample_data


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Score(precision=0.0296627136077912, recall=0.02200184723525876, fmeasure=0.01497359312398335)
Score(precision=0.030836203214894622, recall=0.023598556926424097, fmeasure=0.016039858509563514)
Score(precision=0.0, recall=0.0, fmeasure=0.0)


"A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the\xa0Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the animals shouting 'Today I kill a lion!' Mr Kumar had been sitting near the enclosure when he suddenly made a dash for the lions, surprising zoo security. The intoxicated teenager ran towards the lions, shouting: 'Today I kill a lion or a lion kills me!' A zoo spokesman said: 'Guards had earlier spotted him close to the enclosure but had no idea he was planing to enter it. 'Fortunately, there are eight moats to cross before getting to where the lions usually are an

In [10]:

"""### TESTING ON FINE-TUNED MODEL FROM HUGGING FACE"""

import datasets
from transformers import BertTokenizer, EncoderDecoderModel

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail").to("cuda")
tokenizer = BertTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

test_dataset = Dataset.from_pandas(test_data)

test_dataset = test_dataset.select(range(200))

batch_size = 8

def generate_summary(batch):

    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)


    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_dataset.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]
label_str = results["highlights"]

rougeL_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid
rouge1_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
rouge2_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rougeL_output)
print(rouge1_output)
print(rouge2_output)

results[1]

test_dataset['article'][1]

pred_str

config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Score(precision=0.2589453691579804, recall=0.2961255947705103, fmeasure=0.2696917098397658)
Score(precision=0.38658994430131555, recall=0.43820404831275395, fmeasure=0.4011642807550139)
Score(precision=0.17384853191324717, recall=0.1934675016936152, fmeasure=0.17861216601906998)


["the government is happy to set standards for animals flying on planes. but the government doesn't stipulate a minimum amount of space for humans. many economy seats on united airlines have 30 inches of room. some airlines offer as little as 28 inches, while some airlines have just 28 inches.",
 "rahul kumar, 17, clambered over enclosure fence at kamla nehru zoological park in western india. he began running towards the lions, shouting he would'kill them'before jumping into the enclosure. mr kumar said he was drunk and'thought i'd stand a good chance'against predators.",
 "dougie freedman is set to sign a new deal at nottingham forest. freedman has impressed at the city ground since replacing stuart pearce in february. the club's owners are pleased with freedman's job at the club. click here for all the latest transfer news with our live updates.",
 "liverpool were linked with a move for the 25 - year - old. neto is also wanted by psg and psg in spain. the brazilian's agent has reveal

In [11]:
!pip install datasets rouge_score sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.1 MB/s eta 0:00:00


In [12]:
import datasets
from datasets import load_metric

# Load the metrics
rouge = load_metric("rouge")
bleu = load_metric("sacrebleu")

def compute_metrics(predictions, references):
    # Compute ROUGE scores
    rouge_output = rouge.compute(predictions=predictions, references=references)
    # Prepare data for BLEU
    bleu_output = bleu.compute(predictions=[predictions], references=[[references]])

    # Extracting necessary values for display
    rouge_result = {key: value.mid.fmeasure for key, value in rouge_output.items()}  # taking mid Fmeasure value
    bleu_result = bleu_output['score']

    return {
        "rouge": rouge_result,
        "bleu": bleu_result
    }

# Assuming `results` is your dataset with 'pred' as predicted summaries and 'highlights' as references
pred_str = results["pred"]
label_str = results["highlights"]

# Compute metrics
metrics = compute_metrics(pred_str, label_str)
print(metrics)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from t

{'rouge': {'rouge1': 0.40074590619989864, 'rouge2': 0.17867402149130923, 'rougeL': 0.26984492668082993, 'rougeLsum': 0.338528944968955}, 'bleu': 14.404937496073263}
